## Método de Runge–Kutta de 4ª Ordem (RK4)

### Problema de Valor Inicial (PVI)

Queremos resolver um problema de valor inicial da forma

$$
\frac{dy}{dt} = f(t, y),
\qquad
y(t_0) = y_0
$$

---

### Método RK4

O método de Runge–Kutta de quarta ordem avança a solução do instante
$t_n$ para $t_{n+1} = t_n + h$ por meio de quatro avaliações do campo de derivadas.

As inclinações intermediárias são definidas como:

$$
k_1 = f(t_n, y_n)
$$

$$
k_2 = f\left(t_n + \frac{h}{2},\; y_n + \frac{h}{2}k_1\right)
$$

$$
k_3 = f\left(t_n + \frac{h}{2},\; y_n + \frac{h}{2}k_2\right)
$$

$$
k_4 = f\left(t_n + h,\; y_n + hk_3\right)
$$

A atualização da solução é dada por

$$
y_{n+1}
=
y_n
+
\frac{h}{6}
\left(
k_1 + 2k_2 + 2k_3 + k_4
\right)
$$

---

### Interpretação Numérica

O método RK4 aproxima a solução exata

$$
y(t_{n+1})
=
y(t_n)
+
\int_{t_n}^{t_{n+1}} f(t, y(t))\,dt
$$

por meio de uma combinação ponderada de inclinações avaliadas em pontos internos do intervalo de integração.

---

### Propriedades Numéricas

O RK4 é um método explícito de passo único (one-step method), no qual cada valor
$y_{n+1}$ depende apenas de $(t_n, y_n)$ e das avaliações da função $f$ naquele passo.

O erro local de truncamento é da ordem

$$
O(h^5)
$$

enquanto o erro global acumulado ao longo do intervalo é da ordem

$$
O(h^4)
$$

---

### Generalização para Sistemas

O método se aplica naturalmente a sistemas de equações diferenciais ordinárias da forma

$$
\frac{d\mathbf{y}}{dt} = \mathbf{f}(t, \mathbf{y}),
\qquad
\mathbf{y}(t_0) = \mathbf{y}_0
$$

onde $\mathbf{y} \in \mathbb{R}^n$ representa o vetor de variáveis do sistema.

---

### Considerações de Implementação

Na implementação computacional, utiliza-se uma função genérica que representa o campo vetorial

$$
f : (t, y) \mapsto \frac{dy}{dt}
$$

com validações simples para garantir

$$
h > 0,
\qquad
t_f > t_0
$$

Para eficiência e organização, os vetores de tempo e solução são pré-alocados:

$$
t = \{t_0, t_1, \dots, t_N\},
\qquad
y = \{y_0, y_1, \dots, y_N\}
$$

A interface conceitual do método pode ser expressa como

$$
(t, y) = \texttt{rk4\_solve}(f, (t_0, t_f), y_0, h)
$$


In [1]:
import numpy as np
from typing import Callable, Tuple, Union

ArrayLike = Union[float, np.ndarray]

def rk4_step(f: Callable[[float, ArrayLike], ArrayLike],
             t: float,
             y: ArrayLike,
             h: float) -> ArrayLike:
    """
    Um passo do RK4 clássico.

    y_{n+1} = y_n + (h/6)*(k1 + 2k2 + 2k3 + k4)
    onde:
      k1 = f(t, y)
      k2 = f(t + h/2, y + (h/2)*k1)
      k3 = f(t + h/2, y + (h/2)*k2)
      k4 = f(t + h,   y + h*k3)
    """
    k1 = f(t, y)
    k2 = f(t + 0.5*h, y + 0.5*h*k1)
    k3 = f(t + 0.5*h, y + 0.5*h*k2)
    k4 = f(t + h,     y + h*k3)
    return y + (h / 6.0) * (k1 + 2.0*k2 + 2.0*k3 + k4)


def rk4_solve(f: Callable[[float, ArrayLike], ArrayLike],
              t_span: Tuple[float, float],
              y0: ArrayLike,
              h: float) -> Tuple[np.ndarray, np.ndarray]:
    """
    Resolve y' = f(t, y) em [t0, tf] com passo fixo h usando RK4.

    Retorna:
      t: (N+1,)
      y: (N+1,) se y0 escalar, ou (N+1, dim) se y0 vetor.
    """
    t0, tf = t_span
    if h <= 0:
        raise ValueError("h deve ser > 0.")
    if tf <= t0:
        raise ValueError("t_span deve satisfazer tf > t0.")

    # número de passos (forçamos inclusão de tf com um último passo menor se necessário)
    n_steps = int(np.floor((tf - t0) / h))
    remainder = (tf - t0) - n_steps*h

    # garante y como array (mas mantém compatível com escalar)
    y0_arr = np.array(y0, dtype=float)
    is_scalar = (y0_arr.ndim == 0)

    # pré-alocação: pode ter +1 passo se remainder > 0
    extra = 1 if remainder > 1e-15 else 0
    N = n_steps + extra

    t = np.empty(N + 1, dtype=float)
    y = np.empty((N + 1,) + (() if is_scalar else y0_arr.shape), dtype=float)

    t[0] = t0
    y[0] = y0_arr

    ti = t0
    yi = y0_arr

    for i in range(N):
        # passo final ajustado para cair exatamente em tf
        hi = h if (i < n_steps) else remainder
        yi = rk4_step(f, ti, yi, hi)
        ti = ti + hi

        t[i + 1] = ti
        y[i + 1] = yi

    return t, y


In [3]:
import numpy as np

# 1) defina a EDO
k = 2.0
def f(t, y):
    return -k * y

# 2) parâmetros do problema
t_span = (0.0, 5.0)
y0 = 1.0
h = 0.01

# 3) resolva
t, y = rk4_solve(f, t_span, y0, h)

# y é 1D (N+1,)
print(t[:5])
print(y[:5])
print("y(5) ~", y[-1])


[0.   0.01 0.02 0.03 0.04]
[1.         0.98019867 0.96078944 0.94176453 0.92311635]
y(5) ~ 4.5399930377993055e-05
